In [19]:
import numpy as np
a = 6378137
e2 = 0.00669438002290

def vincenty(BA,LA,BB,LB):
    '''
    Parameters
    ----------
    BA : szerokosc geodezyjna punktu A [RADIAN]
    LA : dlugosc geodezyjna punktu A [RADIAN]
    BB : szerokosc geodezyjna punktu B [RADIAN]
    LB : dlugosc geodezyjna punktu B [RADIAN]

    Returns
    -------
    sAB : dlugosc linii geodezyjnej AB [METR]
    A_AB : azymut linii geodezyjnej AB [RADIAN]
    A_BA : azymut odwrotny linii geodezyjne [RADIAN]
    '''
    b = a * np.sqrt(1-e2)
    f = 1-b/a
    dL = LB - LA
    UA = np.arctan((1-f)*np.tan(BA))
    UB = np.arctan((1-f)*np.tan(BB))
    L = dL
    while True:
        sin_sig = np.sqrt((np.cos(UB)*np.sin(L))**2 +\
                          (np.cos(UA)*np.sin(UB) - np.sin(UA)*np.cos(UB)*np.cos(L))**2)
        cos_sig = np.sin(UA)*np.sin(UB) + np.cos(UA) * np.cos(UB) * np.cos(L)
        sig = np.arctan2(sin_sig,cos_sig)
        sin_al = (np.cos(UA)*np.cos(UB)*np.sin(L))/sin_sig
        cos2_al = 1 - sin_al**2
        cos2_sigm = cos_sig - (2 * np.sin(UA) * np.sin(UB))/cos2_al
        C = (f/16) * cos2_al * (4 + f*(4 - 3 * cos2_al))
        Lst = L
        L = dL + (1-C)*f*sin_al*(sig+C*sin_sig*(cos2_sigm+C*cos_sig*(-1 + 2*cos2_sigm**2)))
        if abs(L-Lst)<(0.000001/206265):
            break
    
    u2 = (a**2 - b**2)/(b**2) * cos2_al
    A = 1 + (u2/16384) * (4096 + u2*(-768 + u2 * (320 - 175 * u2)))
    B = u2/1024 * (256 + u2 * (-128 + u2 * (74 - 47 * u2)))
    d_sig = B*sin_sig * (cos2_sigm + 1/4*B*(cos_sig*(-1+2*cos2_sigm**2)\
            - 1/6 *B*cos2_sigm * (-3 + 4*sin_sig**2)*(-3+4*cos2_sigm**2)))
    sAB = b*A*(sig-d_sig)
    A_AB = np.arctan2((np.cos(UB) * np.sin(L)),(np.cos(UA)*np.sin(UB) - np.sin(UA)*np.cos(UB)*np.cos(L)))
    A_BA = np.arctan2((np.cos(UA) * np.sin(L)),(-np.sin(UA)*np.cos(UB) + np.cos(UA)*np.sin(UB)*np.cos(L))) + np.pi
    return sAB, A_AB, A_BA

# dane gr1 nr2
# 50 stopni i 15 minut
# 18 stopni i 15 minut
phi1 = 50.25
lam1 = 18.25

d12 = 40e3
a12 = 0
d23 = 100e3
a23 = 90
d34 = 40e3
a34 = 180
d41 = 100e3
a41 = 270

Obliczenie współrzędne geodezyjne punktów: 2, 3, 4, z wykorzystaniem biblioteki pyproj

In [20]:
import pyproj
geod = pyproj.Geod(ellps='GRS80')
p2 = geod.fwd(lam1,phi1,a12,d12)
print(p2)
p3 = geod.fwd(p2[0],p2[1],a23,d23)
p4 = geod.fwd(p3[0],p3[1],a34,d34)
p5 = geod.fwd(p4[0],p4[1],a41,d41)

(18.25, 50.609591494809024, 180.0)


In [21]:
# Czy po obliczeniu kolejnych wierzchołków ‘trapezu’, na podstawie podanych obserwacji,
# zamkniemy otrzymamy figurę zamkniętą? Jaka będzie różnica położenia punktów 1 i 1*?
# Czy spowodowana będzie otrzymana różnica?

s_AB, A_AB, A_BA = vincenty(np.deg2rad(phi1),np.deg2rad(lam1),np.deg2rad(p5[1]),np.deg2rad(p5[0]))
print(f"Długość linii 1-1*: {s_AB:.3f} m")


Długość linii 1-1*: 2046.601 m


Różnica położenia punktu 1 i 1* wynika ze sferyczności Ziemi.

In [22]:
# Wyznacz właściwe obserwacje: odległość oraz azymut z punktu 4 do punktu 1 (zadanie
# odwrotne – algorytm Vincentego lub inny),

s_41, A_41, A_14 = vincenty(np.deg2rad(p4[1]),np.deg2rad(p4[0]),np.deg2rad(phi1),np.deg2rad(lam1))
print(f"Długość linii 4-1: {s_41:.3f} m")
az = A_41 if A_41 > 0 else A_41 + 360
stopnie = int(az)
minuty = int((az-stopnie)*60)
sekundy = ((az-stopnie)*60-minuty)*60
print(f"Azymut 4-1: {stopnie}°{minuty}'{sekundy:.5f}\"")

Długość linii 4-1: 100760.090 m
Azymut 4-1: 358°26'53.28649"


In [23]:
# Przedstaw na mapie położenie wszystkich punktów (narysuj powstałą figurę)

import folium
loc = [(phi1+p3[1])/2, (lam1+p3[0])/2]
m = folium.Map(location=loc, zoom_start=9)

folium.Marker([phi1,lam1], popup='P1').add_to(m)
folium.Marker([p2[1],p2[0]], popup='P2').add_to(m)
folium.Marker([p3[1],p3[0]], popup='P3').add_to(m)
folium.Marker([p4[1],p4[0]], popup='P4').add_to(m)
folium.Marker([p5[1],p5[0]], popup='P1*').add_to(m)

folium.PolyLine([[phi1,lam1],[p2[1],p2[0]],[p3[1],p3[0]],[p4[1],p4[0]],[p5[1],p5[0]]], color='red').add_to(m)

m

In [24]:
# Oblicz pole powierzchni powstałej figury (skorzystaj z funkcji z bibliotek języka python, np.
# geometry_area_perimeter lub polygon_area_perimeter z biblioteki pyproj,
# modułu Geod).

from shapely.geometry import Polygon
poly = Polygon([[phi1,lam1],[p2[1],p2[0]],[p3[1],p3[0]],[p4[1],p4[0]],[p5[1],p5[0]]])
area = geod.geometry_area_perimeter(poly)[0]

print(f"Pole powierzchni powstałej figury wynosi {area:.2f} m2")

Pole powierzchni powstałej figury wynosi 6061613983.01 m2
